In [1]:
from infocoupling_FIR_main_nonipype import decode_FIR_timecourses, load_univar_ts, save_univar_ts
import os
from glob import glob
from statsmodels.tsa.stattools import grangercausalitytests
import pandas as pd
import numpy as np
import seaborn as sns
import pickle as pkl
import pingouin as pg

%load_ext autoreload
%autoreload 2

/home/predatt/giaald/.conda/envs/giacomo37/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.0, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/home/predatt/giaald/.conda/envs/giacomo37/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [48]:
def zero_pad(arr):
    ncols = arr.shape[1]
    padding = np.zeros((1, ncols))
    return np.vstack((arr, padding))

In [5]:
multivarfiles = sorted(glob('/project/3018040.05/FIR_timeseries/decoding/test_m29/*_allruns.csv'))
multivarfiles = [m for m in multivarfiles if 'sub-006' not in m]
univarfiles = sorted(glob('/project/3018040.05/FIR_timeseries/univariate/test_m30/*.pkl'))
univarfiles = [u for u in univarfiles if 'sub-006' not in u]

In [41]:
all_multivar_ts = []
for m in multivarfiles:
    all_multivar_ts.append(pd.read_csv(m))
all_multivar_ts = pd.concat(all_multivar_ts)
all_multivar_ts = all_multivar_ts.groupby(['subject', 'runno', 'delay', 'expected']).mean().reset_index()
all_multivar_ts = all_multivar_ts.rename(columns={'runno': 'run'})

In [39]:
all_univar_ts = []
for u in univarfiles:
    all_univar_ts.append(pd.read_pickle(u))
all_univar_ts = pd.concat(all_univar_ts)
all_univar_ts = all_univar_ts.groupby(['subject', 'run', 'delay', 'expected']).mean(numeric_only=False).reset_index()

/home/predatt/giaald/.conda/envs/giacomo37/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [72]:
subjlist = [f'sub-{i:03d}' for i in range(1, 36) if i != 6]
f_differences = []
for sub in subjlist:
    thissub_mv = all_multivar_ts[all_multivar_ts['subject']==sub]
    thissub_uv = all_univar_ts[all_univar_ts['subject']==sub]
    
    ts_uv2mv_exp = [] # "Feedback" univariate --> multivariate
    ts_uv2mv_unexp = []
    ts_mv2uv_exp = [] # "Feedforward" multivariate --> univariate
    ts_mv2uv_unexp = []
    
    for run in range(1, 8):
        thisrun_mv = thissub_mv[thissub_mv['run']==run]
        thisrun_uv = thissub_uv[thissub_uv['run']==run]
        
        mv_ts_exp = np.vstack(thisrun_mv[thisrun_mv.expected==True].distance)
        mv_ts_unexp = np.vstack(thisrun_mv[thisrun_mv.expected==False].distance)
        
        uv_ts_exp = np.mean(np.vstack(thisrun_uv[thisrun_uv.expected==True]['samples']), axis=1, keepdims=True)
        uv_ts_unexp = np.mean(np.vstack(thisrun_uv[thisrun_uv.expected==False]['samples']), axis=1, keepdims=True)
        
        if run != 7:
            mv_ts_exp = zero_pad(mv_ts_exp)
            mv_ts_unexp = zero_pad(mv_ts_unexp)
            uv_ts_exp = zero_pad(uv_ts_exp)
            uv_ts_unexp = zero_pad(uv_ts_unexp)
        
        ts_uv2mv_exp.append(np.hstack([uv_ts_exp, mv_ts_exp]))
        ts_uv2mv_unexp.append(np.hstack([uv_ts_unexp, mv_ts_unexp]))
        ts_mv2uv_exp.append(np.hstack([mv_ts_exp, uv_ts_exp]))
        ts_mv2uv_unexp.append(np.hstack([mv_ts_unexp, uv_ts_unexp]))

    ts_uv2mv_exp = np.vstack(ts_uv2mv_exp)
    ts_uv2mv_unexp = np.vstack(ts_uv2mv_unexp)
    ts_mv2uv_exp = np.vstack(ts_mv2uv_exp)
    ts_mv2uv_unexp = np.vstack(ts_mv2uv_unexp)
    
    gc_uv2mv_exp = grangercausalitytests(ts_uv2mv_exp, 1, verbose=False)
    gc_uv2mv_unexp = grangercausalitytests(ts_uv2mv_unexp, 1, verbose=False)
    gc_mv2uv_exp = grangercausalitytests(ts_mv2uv_exp, 1, verbose=False)
    gc_mv2uv_unexp = grangercausalitytests(ts_mv2uv_unexp, 1, verbose=False)
    
    # Compute F-statistic difference between feedback and feedforward
    f_diff_exp = gc_uv2mv_exp[1][0]['ssr_ftest'][0] - gc_mv2uv_exp[1][0]['ssr_ftest'][0]
    f_diff_unexp = gc_uv2mv_unexp[1][0]['ssr_ftest'][0] - gc_mv2uv_unexp[1][0]['ssr_ftest'][0]
    
    f_differences.append({
        'subject': sub,
        'f_diff_exp': f_diff_exp,
        'f_diff_unexp': f_diff_unexp
    })

f_differences = pd.DataFrame(f_differences)

In [76]:
len(f_differences)

34

In [82]:
pg.wilcoxon(f_differences['f_diff_exp'])

,W-val,alternative,p-val,RBC,CLES
Wilcoxon,203.0,two-sided,0.108038,-0.317647,NaN


In [68]:
gc_uv2mv_exp = grangercausalitytests(ts_uv2mv_exp, 1, verbose=False)

In [71]:
gc_uv2mv_exp[1][0]['ssr_ftest'][0]

0.015917328315767757